In [1]:
import numpy as np
from sklearn.model_selection import train_test_split


In [2]:
scores = np.load('/home/ali.lawati/mol-incontext/input/embed/mmcl_attr-chebi-3-epochs300-loop.mistral-7B.scores.npy')
print(scores.shape)
print(scores)

(3297,)
[0.29757096 0.19992366 0.65875881 ... 0.16584918 0.21265482 0.17678382]


In [3]:
embeds = np.load('/home/ali.lawati/mol-incontext/input/embed/mmcl_attr-chebi-3-epochs300-embeds.npz')
seqs = embeds['embeds']
refs = embeds['test_pool']
refs = np.reshape(refs, (refs.shape[0],1,-1))

print (seqs.shape, refs.shape)

seqs_new = np.concatenate((refs, seqs), axis = 1)
seqs_new.shape

(3297, 3, 64) (3297, 1, 64)


(3297, 4, 64)

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
import pdb

class LSTMWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTMWithAttention, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.attention_layer = nn.Linear(hidden_size, 1)  # To compute attention weights

    def forward(self, x):
        out, (h, n) = self.lstm(x)
        attn_weights = F.softmax(self.attention_layer(out), dim=1)  # Normalize to get attention weights
        context_vector = torch.sum(attn_weights * out, dim=1)  
        print(context_vector)
        combined = torch.cat((context_vector, h[-1]), dim=1)
        score = self.fc(combined)
        return score

In [17]:
from torch.utils.data import DataLoader, TensorDataset, random_split
batch_size = 32
x_tensor = torch.tensor(seqs_new)
y_tensor = torch.tensor(scores)
dataset = TensorDataset(x_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset)-train_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [18]:
input_size = 64
hidden_size = 64
num_layers = 2
epochs = 100
batch_size = 16
learning_rate = 0.0007

model = LSTMWithAttention(input_size, hidden_size)

criterion = nn.MSELoss()  # Mean Squared Error Loss
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    # Training phase
    model.train()
    train_loss = 0.0
    for batch in train_loader:
        optimizer.zero_grad()
        x, y = batch
        #print(x, y)
        outputs = model(x) 
        loss = criterion(outputs, y.float())
        #print(y, loss)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    train_loss /= len(train_loader)
        
    # Validation phase
    model.eval()
    with torch.no_grad():  # No gradients needed for validation
        val_loss = 0.0
        for batch in val_loader:
            x, y = batch
            outputs = model(x)  # Outputs shape: [batch_size, output_size]
            loss = criterion(outputs, y)
            val_loss += loss.item()

    val_loss /= len(val_loader)
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")


tensor([[-0.0245,  0.0496,  0.0357,  ..., -0.0916, -0.0377, -0.0325],
        [-0.0277,  0.0408,  0.0263,  ..., -0.0819, -0.0706, -0.0380],
        [-0.0050,  0.0523,  0.0463,  ..., -0.0780, -0.0413, -0.0376],
        ...,
        [-0.0167,  0.0304,  0.0348,  ..., -0.0958, -0.0343, -0.0378],
        [ 0.0061,  0.0323,  0.0468,  ..., -0.0910, -0.0517, -0.0452],
        [-0.0183,  0.0317,  0.0565,  ..., -0.0716, -0.0489, -0.0246]],
       grad_fn=<SumBackward1>)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x128 and 64x1)

In [152]:
output = model(x_tensor)

In [165]:
output = output.squeeze(1) #topk(10)
output.shape

torch.Size([3297])

In [167]:
output.topk(100)

torch.return_types.topk(
values=tensor([0.4076, 0.4037, 0.4013, 0.3975, 0.3969, 0.3967, 0.3965, 0.3955, 0.3930,
        0.3928, 0.3909, 0.3906, 0.3900, 0.3894, 0.3880, 0.3878, 0.3873, 0.3860,
        0.3856, 0.3852, 0.3849, 0.3848, 0.3839, 0.3831, 0.3830, 0.3829, 0.3822,
        0.3816, 0.3807, 0.3807, 0.3795, 0.3790, 0.3783, 0.3779, 0.3779, 0.3771,
        0.3760, 0.3758, 0.3757, 0.3751, 0.3746, 0.3742, 0.3740, 0.3738, 0.3737,
        0.3736, 0.3735, 0.3734, 0.3734, 0.3729, 0.3726, 0.3725, 0.3716, 0.3715,
        0.3715, 0.3715, 0.3713, 0.3711, 0.3710, 0.3705, 0.3704, 0.3702, 0.3701,
        0.3699, 0.3699, 0.3698, 0.3697, 0.3697, 0.3696, 0.3691, 0.3690, 0.3688,
        0.3688, 0.3685, 0.3684, 0.3684, 0.3684, 0.3683, 0.3679, 0.3679, 0.3678,
        0.3678, 0.3673, 0.3672, 0.3672, 0.3671, 0.3666, 0.3666, 0.3666, 0.3665,
        0.3665, 0.3663, 0.3662, 0.3662, 0.3661, 0.3661, 0.3661, 0.3660, 0.3660,
        0.3659], grad_fn=<TopkBackward0>),
indices=tensor([ 557,  963, 1522, 1666, 2918,